# Debugging Langchain
## Introduction

Abstractions are great, but sometimes we want to see what goes on under the hood of langchain. 
Here's a few options:

- https://python.langchain.com/docs/guides/debugging
- https://python.langchain.com/docs/modules/chains/how_to/debugging

## Installation

In [1]:
%pip install -q langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


## Global debugging flag
First option is to enable global debugging in Langchain.

You can see , it shows a lot of the calls/events that go on internally.

In [2]:
import langchain
from langchain_openai import ChatOpenAI

# Enable global debugging
langchain.debug = True

llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)
llm.invoke("Where is Ghent ?")
# Disable global debugging
langchain.debug = False

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Where is Ghent ?"
  ]
}
[llm/end] [llm:ChatOpenAI] [2.26s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ghent is a city located in Belgium, situated in the Flanders region. It is the capital of the East Flanders province and is known for its rich history, medieval architecture, and vibrant cultural scene. Ghent is located at the confluence of the Rivers Scheldt and Lys, making it an important historical trade center. The city is also home to several notable landmarks, including the Gravensteen castle, St. Bavo's Cathedral, and the picturesque Graslei and Korenlei streets along the river.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
      

## Increase verbosity
Second one is to tell providers or objects to be more verbose

In [3]:
# Set verbose flag on objects
llm = ChatOpenAI(model="gpt-4o-mini",verbose=True, temperature=0)
result = llm.invoke("Where is Ghent?")
print(result.content)

Ghent is a city located in Belgium, specifically in the Flanders region. It is situated at the confluence of the Rivers Scheldt and Lys. Ghent is known for its rich history, medieval architecture, and vibrant cultural scene. It is one of the largest cities in Belgium and serves as the capital of the East Flanders province. The city is also famous for its university, which is one of the oldest and most prestigious in the country.


Now without the verbose flag. As you can see , there is a not a lot of difference.

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini",verbose=False, temperature=0)
result = llm.invoke("Where is Ghent?")
print(result.content)

Ghent is a city located in Belgium, specifically in the Flanders region. It is situated at the confluence of the Rivers Scheldt and Lys. Ghent is known for its rich history, medieval architecture, and vibrant cultural scene. It is one of the largest cities in Belgium and serves as the capital of the East Flanders province. The city is also famous for its university, which is one of the oldest and most prestigious in the country.


## Using callbacks
A third useful way is to use callbacks to listen into objects events. We created our own helper in these lessons to listen to events.

In [5]:
from langchain_core.tracers.stdout import ConsoleCallbackHandler

# A console callback handler
console_callback = ConsoleCallbackHandler()

# Added to the model as callback option
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0 , callbacks=[console_callback])

result = llm.invoke("Where is Ghent?")
print(result.content)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Where is Ghent?"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ghent is a city located in Belgium, specifically in the Flanders region. It is situated at the confluence of the Rivers Scheldt and Lys. Ghent is known for its rich history, medieval architecture, and vibrant cultural scene. It is one of the largest cities in Belgium and serves as the capital of the East Flanders province. The city is also famous for its university, which is one of the oldest and most prestigious in the country.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
         

## Callback for LCEL syntax
We also use the callback handler with the LCEL syntax

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

# Adding the callback
callbacks = [ConsoleCallbackHandler()]
chain.invoke({"topic": "ice cream"}, config={'callbacks': callbacks})


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "topic": "ice cream"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "ice cream"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a joke about ice cream"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Why did the ice cream truck break down?\n\nBecause it had a rocky road!",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
         

'Why did the ice cream truck break down?\n\nBecause it had a rocky road!'